In [1]:
import pandas as pd

DATA_FILE = '../../data/chimpanzees.csv'

In [2]:
df = pd.read_csv(DATA_FILE)
df.head()

,actor,recipient,condition,block,trial,prosoc_left,chose_prosoc,pulled_left
0,1,NaN,0,1,2,0,1,0
1,1,NaN,0,1,4,0,0,1
2,1,NaN,0,1,6,1,0,0
3,1,NaN,0,1,8,0,1,0
4,1,NaN,0,1,10,1,1,1


### Code 11.2

In [3]:
df['treatment'] = 1 + df['prosoc_left'] + 2 * df['condition']
df[['condition', 'prosoc_left', 'treatment']].drop_duplicates()

,condition,prosoc_left,treatment
0,0,0,1
2,0,1,2
36,1,0,3
38,1,1,4


In [5]:
df['actor'].drop_duplicates()

0      1
72     2
144    3
216    4
288    5
360    6
432    7
Name: actor, dtype: int64

## Model

\begin{align*}
    L_i & \sim \text{Binomial}(1, p_i) \\
    \log \frac{p_i}{1 - p_i} & = \alpha_{\text{actor}[i]} + \beta_{\text{treatment}[i]} \\
    \alpha_j & \sim \text{to be determined} \\
    \beta_j & \sim \text{to be determined}
\end{align*}

Here $L$ indicates the binary variable `pulled_left`; there are $7$ `$\alpha$ parameters`, one for each chimpanzee; there are $4$ `treatment` variables, one for each combination of the position of the pro-social option and the presence/absence of a partner.

## First Attempt Using Only $\alpha$

The very first attempt consists of using a flat prior for $\alpha$.

\begin{align*}
    L_i & \sim \text{Binomial}(1, p_i) \\
    \log \frac{p_i}{1 - p_i} & = \alpha  \\
    \alpha & \sim \text{Normal}(0, \omega)
\end{align*}


In [7]:
import arviz as az
import pymc3 as pm
import numpy as np
import matplotlib.pyplot as plt

from scipy import stats
from scipy.special import expit as logistic

In [ ]:
with pm.Model() as m_11_1:
    a = pm.Normal('a', mu=0, sigma=10)
    